# Neural Network Training — NumPy From Scratch

A minimal 2-layer neural network with manual forward/backprop and gradient checking.

In [ ]:

import numpy as np

# Set random seed
np.random.seed(42)

# Generate synthetic data
def make_data(n=400):
    X = np.random.randn(n, 2)
    y = (X[:,0]*X[:,1] > 0).astype(int).reshape(-1,1)
    return X, y

X, y = make_data()
print(X.shape, y.shape)


In [ ]:

# Initialize weights
n_input, n_hidden, n_output = 2, 5, 1
W1 = 0.01 * np.random.randn(n_input, n_hidden)
b1 = np.zeros((1, n_hidden))
W2 = 0.01 * np.random.randn(n_hidden, n_output)
b2 = np.zeros((1, n_output))

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def sigmoid_deriv(a):
    return a * (1 - a)


In [ ]:

# Training loop (basic)
lr = 0.1
for epoch in range(2000):
    # Forward
    z1 = X.dot(W1) + b1
    a1 = np.maximum(0, z1)  # ReLU
    z2 = a1.dot(W2) + b2
    y_hat = sigmoid(z2)
    loss = -(y*np.log(y_hat+1e-8) + (1-y)*np.log(1-y_hat+1e-8)).mean()
    
    # Backprop
    dz2 = y_hat - y
    dW2 = a1.T.dot(dz2) / len(X)
    db2 = dz2.mean(axis=0, keepdims=True)
    da1 = dz2.dot(W2.T)
    dz1 = da1 * (z1 > 0)
    dW1 = X.T.dot(dz1) / len(X)
    db1 = dz1.mean(axis=0, keepdims=True)
    
    # Update
    W1 -= lr * dW1
    b1 -= lr * db1
    W2 -= lr * dW2
    b2 -= lr * db2
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, loss {loss:.4f}")


In [ ]:

# Gradient check example
eps = 1e-5
ix = (0,0)
oldval = W1[ix]
W1[ix] = oldval + eps
z1 = X.dot(W1) + b1
a1 = np.maximum(0, z1)
z2 = a1.dot(W2) + b2
y_hat = sigmoid(z2)
loss_plus = -(y*np.log(y_hat+1e-8)+(1-y)*np.log(1-y_hat+1e-8)).mean()

W1[ix] = oldval - eps
z1 = X.dot(W1) + b1
a1 = np.maximum(0, z1)
z2 = a1.dot(W2) + b2
y_hat = sigmoid(z2)
loss_minus = -(y*np.log(y_hat+1e-8)+(1-y)*np.log(1-y_hat+1e-8)).mean()

W1[ix] = oldval
num_grad = (loss_plus - loss_minus)/(2*eps)
print("Numerical grad:", num_grad)
